In [1]:
import data

In [2]:
from smackbang.matches import get_matches

In [3]:
from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
# Authentication
consumerKey = "PqA88zMUxSwVZi5hJpKdVKT4L"
consumerSecret = "WQ1xDXrygO4US93heMenGkj5z6DZeUQrZDIyoACAVZqtzulpPc"
accessToken = "1499085994561310727-p9J1gRV1tsLrVowGCvinw3lEvMHB5z"
accessTokenSecret = "zVxDSNXC8qmv2ZR5ZD6bE7rT0EmaRewTJxakYlQx89ncL"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [5]:
#Sentiment Analysis

def analyze_tweet(cities):
    noOfTweet = 2000
    list_df= []
    for city in cities:      

        #tweets = tweepy.Cursor(api.search_tweets(), q=keyword).items(noOfTweet)
        tweets = api.search_tweets(q=city, count=noOfTweet)

        tweet_list = []

        for tweet in tweets:
            tweet_list.append(tweet.text)

        #Tweets to DF
        tweet_list = pd.DataFrame(tweet_list)

        #Clean the data
        tweet_list.drop_duplicates(inplace = True)
        tw_list = pd.DataFrame(tweet_list)

        #Make another column to compare
        tw_list["text"] = tw_list[0]

        #More cleaning
        remove_rt = lambda x: re.sub('RT @\w+:'," ",x)
        rt = lambda x: re.sub('(@[A-Za-z0–9]+)|(\w+:\/\/\S+)'," ",x)
        removen = lambda x: re.sub('\n',' ', x)
        tw_list["text"] = tw_list.text.map(remove_rt).map(rt).map(removen)
        tw_list["text"] = tw_list.text.str.lower()

        #I will try to simplify this
        tw_list[["polarity", "subjectivity"]] = tw_list["text"].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        for index, row in tw_list["text"].iteritems():
            score = SentimentIntensityAnalyzer().polarity_scores(row)
            neg = score["neg"]
            neu = score["neu"]
            pos = score["pos"]
            comp = score["compound"]
            if neg > pos:
                tw_list.loc[index, "sentiment"] = "negative"
            elif pos > neg:
                tw_list.loc[index, "sentiment"] = "positive"
            else:
                tw_list.loc[index, "sentiment"] = "neutral"
                tw_list.loc[index, "neg"] = neg
                tw_list.loc[index, "neu"] = neu
                tw_list.loc[index, "pos"] = pos
                tw_list.loc[index, "compound"] = comp

        def thumb(neg,neu,pos):
            if pos+(neu/2) >= neg+(neu/2):
                return "👍"
            else:
                return "👎"

        #Output a simplified DF
        def count_values_in_column(data,feature):
            
            total=data.loc[:,feature].value_counts(dropna=False)
            percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
            df= pd.concat([total,percentage],axis=1,keys=["Total","Percentage"])
            df["City"] = city
            df["Verdict"] = thumb(df["Total"].loc["negative"], df["Total"].loc["neutral"], df["Total"].loc["positive"])
            list_df.append(df)


        #Count_values for sentiment
        count_values_in_column(tw_list,"sentiment")
    result =  pd.concat(list_df)
    return result.reset_index(drop=True).drop(columns=["Total","Percentage"]).drop_duplicates()

In [37]:
df = analyze_tweet(["Japan","Russia","Korea"])


In [38]:
df

,City,Verdict
0,Japan,👍
3,Russia,👎
6,Korea,👍


In [41]:
result  = df.to_dict()

In [42]:
pd.DataFrame(result)

,City,Verdict
0,Japan,👍
3,Russia,👎
6,Korea,👍


In [43]:
cities = 'SYD,NRT'

In [44]:
cities.split(',')

['SYD', 'NRT']

In [45]:
from smackbang.locations import get_city_location

In [59]:
result = get_city_location(["AKL","SYD"])

In [60]:
result ={
  "lat": {
    "0": -33.8689,
    "1": -36.84846
  },
  "lon": {
    "0": 151.20709,
    "1": 174.76334
  },
  "city_code": {
    "0": "SYD",
    "1": "AKL"
  }
}

In [61]:
pd.DataFrame(result)

,lat,lon,city_code
0,-33.86890,151.20709,SYD
1,-36.84846,174.76334,AKL
